## **TODO and Notes**

### TODO

- [x] Make dataframes
- [x] Convert datetimes
- [x] rename date columns
- [x] Find Nans
- [x] Re-freq and fill blanks
- [x] Turn logs into daily totals
- [x] Save backup CSVs
- [x] Combine dataframe
- [x] Make plots
- [x] Figure out hovertips
- [x] Refactor saving pattern
 
 ### **Notes**
Sample for iterating through different offsets 
```python
df["Input"].corr(df["Output"].shift(-1), method = 'pearson', min_periods = 1) #1
```
and more iteration 
```python
 xcov_monthly = [crosscorr(datax, datay, lag=i) for i in range(12)]
```
from [here](https://stackoverflow.com/questions/33171413/cross-correlation-time-lag-correlation-with-pandas)


## Imports, data, checks

In [116]:
import numpy as np
import requests
import pandas as pd
import datetime
import colorcet as cc
from jinja2 import Environment, FileSystemLoader
from urllib.request import urlopen
import json
from bokeh.models import CategoricalColorMapper, NumeralTickFormatter, HoverTool
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, VBar
from bokeh.plotting import output_notebook, figure
from bokeh.io import reset_output, show, output_file, save
from bokeh.layouts import column, row
from bokeh.palettes import all_palettes, Turbo256, Inferno256, RdPu, RdYlBu, RdBu
from bokeh.transform import linear_cmap
from bokeh.embed import autoload_static, components, json_item
from bokeh.resources import CDN, INLINE



The vaccine, cases, and deaths source data were relatively easy to grab diretly from the [Larimer county dashboard](https://www.larimer.org/health/communicable-disease/coronavirus-covid-19/larimer-county-positive-covid-19-numbers#/app?tab=risk) as the CSVs download through urls.

In [117]:
larimer_vac_source = pd.read_csv(
    'https://speedtest.larimer.org/covid/index.php?file=vaccinations&csv')

larimer_cases_source = pd.read_csv(
    'https://speedtest.larimer.org/covid/cases.csv', parse_dates=['ReportedDate'])

larimer_deaths_source = pd.read_csv(
    'https://larimer-county-data-lake.s3-us-west-2.amazonaws.com/Public/covid/covid_deaths.csv?t=1631890252549')


The hospitalization data was much more tricky (at least finding a simple solution was tricky) I spent several hours in webscraping research and attempts purgatory. I checked BeautifulSoup, html5lib, lxml, etc. in multiple combinations and none of them had straightforward solutions because the table for hospitalizations is actually rendered through javascript so there is nothing to scrape without actually clicking the buttons. I started down the Selenium and phantomjs path but it seemed like a nightmare. I found this lifesaving article at [Towards Data Science](https://towardsdatascience.com/data-science-skills-web-scraping-javascript-using-python-97a29738353f) which shows how to find specific XHR request urls in the browser developer tools. The requested URL for the rendered table is a pretty vanilla json and not behind any authorization so there is a pretty clean way to get to it. Praise Satan I didn't have to use Selenium.  

In [118]:
url = 'https://larimer-county-data-lake.s3-us-west-2.amazonaws.com/Public/covid/covid_patient_trend.json?t=1632506827395'

response = urlopen(url)
json_data = response.read().decode('utf-8', 'replace')

d = json.loads(json_data)
larimer_hosp_source = pd.json_normalize(d['data'])


In [119]:
# make .csv backups of source data

larimer_vac_source.to_csv('larimer_vac_backup.csv')

larimer_cases_source.to_csv('larimer_cases_backup.csv')

larimer_deaths_source.to_csv('larimer_deaths_backup.csv')

larimer_hosp_source.to_csv('larimer_hosp_backup.csv')


Re-read the backup CSVs so that the notebook runs locally from this point forward.

In [120]:
larimer_vac = pd.read_csv('larimer_vac_backup.csv')

larimer_cases = pd.read_csv('larimer_cases_backup.csv')

larimer_deaths = pd.read_csv('larimer_deaths_backup.csv')

larimer_hosp = pd.read_csv('larimer_hosp_backup.csv')


The source date introduced a random single row from 1970 which is propagating to all the resulting DFs and visualizations so I have to add this dumb piece of code to remove it.

So now we have all of our dataframes

In [121]:
display(larimer_vac)

display(larimer_cases)

display(larimer_deaths)

display(larimer_hosp)


,Unnamed: 0,Date,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
0,0,12/14/2020,32,32,32,32,1,1,0.0,0,...,0.0,0,1,1,0.0,0,NaN,0,0,0
1,1,12/15/2020,13,45,13,45,0,1,NaN,0,...,NaN,0,0,1,0.0,0,NaN,0,0,0
2,2,12/16/2020,303,348,303,348,0,1,0.0,0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
3,3,12/17/2020,990,1338,990,1338,0,1,0.0,0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
4,4,12/18/2020,1057,2395,1057,2395,2,3,0.0,0,...,0.0,0,2,3,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,425,2/12/2022,249,612593,41,256125,51,235535,0.0,34762,...,5.0,13948,39,198635,5.0,14270,2.0,8682,160,137641
426,426,2/13/2022,75,612668,18,256143,16,235551,0.0,34762,...,0.0,13948,10,198645,4.0,14274,2.0,8684,43,137684
427,427,2/14/2022,237,612905,42,256185,51,235602,4.0,34766,...,8.0,13956,37,198682,2.0,14276,4.0,8688,146,137830
428,428,2/15/2022,272,613177,35,256220,82,235684,0.0,34766,...,12.0,13968,58,198740,7.0,14283,5.0,8693,159,137989


,Unnamed: 0,CaseCount,ReportedDate,Sex,Age,Type,City
0,0,1,2020-03-09,Female,52.0,Confirmed,Johnstown
1,1,2,2020-03-15,Male,49.0,Confirmed,Fort Collins
2,2,3,2020-03-17,Female,53.0,Confirmed,Fort Collins
3,3,4,2020-03-17,Female,94.0,Confirmed,Loveland
4,4,5,2020-03-18,Male,49.0,Confirmed,Fort Collins
...,...,...,...,...,...,...,...
74745,74745,77059,2022-02-20,Male,70.0,Confirmed,Loveland
74746,74746,77060,2022-02-20,Male,75.0,Confirmed,Loveland
74747,74747,77061,2023-02-18,Male,2.0,Probable,Berthoud
74748,74748,77062,2023-02-18,Male,21.0,Probable,Fort Collins


,Unnamed: 0,death_id,death_date,age,gender,city,case_status,count
0,0,a0U5w00000edbfjEAA,2020-03-09,91,Female,Loveland,Probable,1
1,1,a0U5w00000edbfiEAA,2020-03-13,95,Female,Loveland,Probable,2
2,2,a0U5w00000edbfOEAQ,2020-03-15,90,Female,Loveland,Probable,3
3,3,a0U5w00000edbfJEAQ,2020-03-25,87,Female,Fort Collins,Confirmed,4
4,4,a0U5w00000edbfMEAQ,2020-03-25,74,Female,Loveland,Confirmed,5
...,...,...,...,...,...,...,...,...
461,461,a0U5w00000iX2PbEAK,2022-02-02,53,Male,Red Feather Lakes,Confirmed,462
462,462,a0U5w00000fpAoDEAU,2022-02-03,90,Female,Estes Park,Confirmed,463
463,463,a0U5w00000fpAoOEAU,2022-02-03,94,Female,Berthoud,Confirmed,464
464,464,a0U5w00000fpBnzEAE,2022-02-05,89,Male,Fort Collins,Confirmed,465


,Unnamed: 0,Date,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
0,0,2020-03-31T00:00:00.000Z,NaN,NaN,47,0,NaN
1,1,2020-04-01T00:00:00.000Z,NaN,NaN,46,0,NaN
2,2,2020-04-02T00:00:00.000Z,NaN,NaN,46,0,NaN
3,3,2020-04-03T00:00:00.000Z,2.0,0.0,46,0,NaN
4,4,2020-04-04T00:00:00.000Z,1.0,0.0,42,0,NaN
...,...,...,...,...,...,...,...
469,469,2022-02-14T00:00:00.000Z,5.0,0.0,43,0,-40.277778
470,470,2022-02-15T00:00:00.000Z,5.0,0.0,39,0,-30.357143
471,471,2022-02-16T00:00:00.000Z,5.0,0.0,40,0,-32.203390
472,472,2022-02-17T00:00:00.000Z,8.0,0.0,40,0,-32.203390


This looks like pretty good start. We'll have to make all the datetimes match and the **hospitalization** and **vaccine** data are daily totals while the **death** and **case counts** data is a case log (a row for each case) so we'll have to do some grouping to get that to match, that will come later.

## Explore, clean, manipulate

In [122]:
dfs = [larimer_vac, larimer_deaths, larimer_cases, larimer_hosp]


def get_obj_col():
    for df in dfs:
        print(list(df.select_dtypes(['object']).columns))


get_obj_col()


['Date']
['death_id', 'death_date', 'gender', 'city', 'case_status']
['ReportedDate', 'Sex', 'Type', 'City']
['Date']


---
I did this and don't like it
```python

dfs = [larimer_vac, larimer_deaths, larimer_cases, larimer_hosp]
df_names = ['larimer_vac', 'larimer_deaths', 'larimer_cases', 'larimer_hosp']


def get_obj_col():
    for df in dfs:
        obj_cols.append(list(df.select_dtypes(['object']).columns))
    zip(df_names, dfs)
    
obj_cols = []
get_obj_col()
zipped_list = zip(df_names, obj_cols)
print(tuple(zipped_list)
```
---

In [123]:
print(larimer_cases.dtypes)
print(larimer_hosp.dtypes)


Unnamed: 0        int64
CaseCount         int64
ReportedDate     object
Sex              object
Age             float64
Type             object
City             object
dtype: object
Unnamed: 0                      int64
Date                           object
admission_count               float64
kpi_admits_indicator          float64
inpatient_count                 int64
kpi_patient_indicator           int64
inpatient_count_pct_change    float64
dtype: object


Convert date columns from each df to datetimes

In [124]:
larimer_vac['Date'] = pd.to_datetime(larimer_vac['Date']).dt.tz_localize(None)
larimer_deaths['Date'] = pd.to_datetime(
    larimer_deaths['death_date']).dt.tz_localize(None)
larimer_cases['Date'] = pd.to_datetime(
    larimer_cases['ReportedDate']).dt.tz_localize(None)
larimer_hosp['Date'] = pd.to_datetime(
    larimer_hosp['Date']).dt.tz_localize(None)


```pd.to_datetime``` was sufficient for most of the dfs but the hospital data was TZ aware and I wanted all of them to match so had to add the ```.dt.tz_localize(None)``` 

In [125]:
def check_date_type():
    for df in dfs:
        print(list(df.select_dtypes(['datetime64']).columns))


check_date_type()


['Date']
['Date']
['Date']
['Date']


The source randomly introduced a dumbass entry from 1970 in the first row which was propogating to all the following DFs and visualizations so I had to remove it here.

In [126]:
larimer_hosp = larimer_hosp[larimer_hosp['Date']
                            > pd.to_datetime('2020-03-30')]
larimer_hosp.head()


,Unnamed: 0,Date,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
0,0,2020-03-31,NaN,NaN,47,0,NaN
1,1,2020-04-01,NaN,NaN,46,0,NaN
2,2,2020-04-02,NaN,NaN,46,0,NaN
3,3,2020-04-03,2.0,0.0,46,0,NaN
4,4,2020-04-04,1.0,0.0,42,0,NaN


In [127]:
# create daily cases from case log
daily_cases = larimer_cases.groupby(['Date']).count().reset_index()

display(daily_cases)
display(daily_cases.dtypes)
print(f"Total case check {daily_cases['CaseCount'].sum()}")
display(daily_cases.describe())


,Date,Unnamed: 0,CaseCount,ReportedDate,Sex,Age,Type,City
0,2020-03-09,1,1,1,1,1,1,1
1,2020-03-15,1,1,1,1,1,1,1
2,2020-03-17,2,2,2,2,2,2,2
3,2020-03-18,1,1,1,1,1,1,1
4,2020-03-19,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...
703,2022-02-17,100,100,100,100,100,100,100
704,2022-02-18,83,83,83,83,83,83,83
705,2022-02-19,47,47,47,47,47,47,47
706,2022-02-20,11,11,11,11,11,11,11


Date            datetime64[ns]
Unnamed: 0               int64
CaseCount                int64
ReportedDate             int64
Sex                      int64
Age                      int64
Type                     int64
City                     int64
dtype: object

Total case check 74750


,Unnamed: 0,CaseCount,ReportedDate,Sex,Age,Type,City
count,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000,708.000000
mean,105.579096,105.579096,105.579096,105.579096,105.422316,105.579096,105.579096
std,144.065258,144.065258,144.065258,144.065258,143.930002,144.065258,144.065258
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,18.750000,18.750000,18.750000,18.750000,18.750000,18.750000,18.750000
50%,66.500000,66.500000,66.500000,66.500000,66.500000,66.500000,66.500000
75%,129.250000,129.250000,129.250000,129.250000,129.000000,129.250000,129.250000
max,1100.000000,1100.000000,1100.000000,1100.000000,1097.000000,1100.000000,1100.000000


In [128]:
# create daily deaths from death log
daily_deaths = larimer_deaths.groupby(['Date']).count().reset_index()

display(daily_deaths)
display(daily_deaths.dtypes)
print(f"Total death check {daily_deaths['count'].sum()}")
display(daily_deaths.describe())


,Date,Unnamed: 0,death_id,death_date,age,gender,city,case_status,count
0,2020-03-09,1,1,1,1,1,1,1,1
1,2020-03-13,1,1,1,1,1,1,1,1
2,2020-03-15,1,1,1,1,1,1,1,1
3,2020-03-25,2,2,2,2,2,2,2,2
4,2020-03-29,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...
274,2022-01-31,2,2,2,2,2,2,2,2
275,2022-02-02,1,1,1,1,1,1,1,1
276,2022-02-03,2,2,2,2,2,2,2,2
277,2022-02-05,1,1,1,1,1,1,1,1


Date           datetime64[ns]
Unnamed: 0              int64
death_id                int64
death_date              int64
age                     int64
gender                  int64
city                    int64
case_status             int64
count                   int64
dtype: object

Total death check 466


,Unnamed: 0,death_id,death_date,age,gender,city,case_status,count
count,279.000000,279.000000,279.000000,279.000000,279.000000,279.000000,279.000000,279.000000
mean,1.670251,1.670251,1.670251,1.670251,1.670251,1.670251,1.670251,1.670251
std,1.075668,1.075668,1.075668,1.075668,1.075668,1.075668,1.075668,1.075668
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


In [129]:
daily_cases.set_index('Date', inplace=True)

daily_deaths.set_index('Date', inplace=True)

larimer_vac.set_index('Date', inplace=True)

larimer_hosp.set_index('Date', inplace=True)


In [130]:
# daily_cases.index = pd.to_datetime(daily_cases.index)
# daily_cases = daily_cases.resample("1D").mean()
# daily_cases


**Try this**

```python
x.dt = pd.to_datetime(x.dt)
```
One-liner using mostly @ayhan's ideas while incorporating stack/unstack and fill_value

```python
x.set_index(
    ['dt', 'user']
).unstack(
    fill_value=0
).asfreq(
    'D', fill_value=0
).stack().sort_index(level=1).reset_index()
```
**or this might be better**
```python
s.asfreq('D'))
```


In [131]:
larimer_hosp['admission_count'] = larimer_hosp['admission_count'].astype(
    "Int64")
larimer_hosp


,Unnamed: 0,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
Date,,,,,,
2020-03-31,0,<NA>,NaN,47,0,NaN
2020-04-01,1,<NA>,NaN,46,0,NaN
2020-04-02,2,<NA>,NaN,46,0,NaN
2020-04-03,3,2,0.0,46,0,NaN
2020-04-04,4,1,0.0,42,0,NaN
...,...,...,...,...,...,...
2022-02-14,469,5,0.0,43,0,-40.277778
2022-02-15,470,5,0.0,39,0,-30.357143
2022-02-16,471,5,0.0,40,0,-32.203390


In [132]:
larimer_hosp[larimer_hosp.index.duplicated()]


,Unnamed: 0,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
Date,,,,,,
2021-12-15,428,6,0.0,81,1,5.194805
2021-12-15,429,6,0.0,81,1,8.000000
2021-12-15,430,6,0.0,81,1,5.194805


This weird 'Unamed:0" column appeared when I switched to using dfs from the backup CSVs so I had to drop it in place to make the following duplicate drops work.


In [133]:
larimer_hosp.drop(['Unnamed: 0'], axis=1, inplace=True)


In [134]:
larimer_hosp.drop_duplicates(keep=False, inplace=True)


In [135]:
larimer_hosp[larimer_hosp.index.duplicated()]


,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
Date,,,,,


In [136]:
daily_cases_filled = daily_cases.asfreq('D', fill_value=0)
daily_deaths_filled = daily_deaths.asfreq('D', fill_value=0)
larimer_vac_filled = larimer_vac.asfreq('D', fill_value=0)
larimer_hosp_filled = larimer_hosp.asfreq('D', fill_value=0)


## Quantify missing data

In [137]:
print(daily_cases_filled.isna().sum().sum())
print(daily_deaths_filled .isna().sum().sum())
print(larimer_vac_filled .isna().sum().sum())
print(larimer_hosp_filled.isna().sum().sum())


0
0
20
38


In [138]:
larimer_hosp_filled = larimer_hosp_filled.fillna(0)
larimer_vac_filled = larimer_vac_filled.fillna(0)


In [139]:
print(daily_cases_filled.isna().sum().sum())
print(daily_deaths_filled .isna().sum().sum())
print(larimer_vac_filled .isna().sum().sum())
print(larimer_hosp_filled.isna().sum().sum())


0
0
0
0


In [140]:
display(daily_cases_filled)
display(daily_deaths_filled)
display(larimer_vac_filled)
display(larimer_hosp_filled)


,Unnamed: 0,CaseCount,ReportedDate,Sex,Age,Type,City
Date,,,,,,,
2020-03-09,1,1,1,1,1,1,1
2020-03-10,0,0,0,0,0,0,0
2020-03-11,0,0,0,0,0,0,0
2020-03-12,0,0,0,0,0,0,0
2020-03-13,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2023-02-14,0,0,0,0,0,0,0
2023-02-15,0,0,0,0,0,0,0
2023-02-16,0,0,0,0,0,0,0


,Unnamed: 0,death_id,death_date,age,gender,city,case_status,count
Date,,,,,,,,
2020-03-09,1,1,1,1,1,1,1,1
2020-03-10,0,0,0,0,0,0,0,0
2020-03-11,0,0,0,0,0,0,0,0
2020-03-12,0,0,0,0,0,0,0,0
2020-03-13,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
2022-02-02,1,1,1,1,1,1,1,1
2022-02-03,2,2,2,2,2,2,2,2
2022-02-04,0,0,0,0,0,0,0,0


,Unnamed: 0,daily number of doses received by Larimer County residents,total number of doses recevied by residents,daily number of residents receiving first dose,total number of residents receiving first dose,daily number of residents vaccinated,total number of residents vaccinated,daily number of 70+ vaccinated,total number of 70+ vaccinated,daily number of 70+ at least one dose,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-14,0,32,32,32,32,1,1,0.0,0,1.0,...,0.0,0,1,1,0.0,0,0.0,0,0,0
2020-12-15,1,13,45,13,45,0,1,0.0,0,0.0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
2020-12-16,2,303,348,303,348,0,1,0.0,0,2.0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
2020-12-17,3,990,1338,990,1338,0,1,0.0,0,11.0,...,0.0,0,0,1,0.0,0,0.0,0,0,0
2020-12-18,4,1057,2395,1057,2395,2,3,0.0,0,15.0,...,0.0,0,2,3,0.0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-12,425,249,612593,41,256125,51,235535,0.0,34762,0.0,...,5.0,13948,39,198635,5.0,14270,2.0,8682,160,137641
2022-02-13,426,75,612668,18,256143,16,235551,0.0,34762,1.0,...,0.0,13948,10,198645,4.0,14274,2.0,8684,43,137684
2022-02-14,427,237,612905,42,256185,51,235602,4.0,34766,2.0,...,8.0,13956,37,198682,2.0,14276,4.0,8688,146,137830


,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
Date,,,,,
2020-03-31,0,0.0,47,0,0.000000
2020-04-01,0,0.0,0,0,0.000000
2020-04-02,0,0.0,0,0,0.000000
2020-04-03,2,0.0,46,0,0.000000
2020-04-04,1,0.0,42,0,0.000000
...,...,...,...,...,...
2022-02-14,5,0.0,43,0,-40.277778
2022-02-15,5,0.0,39,0,-30.357143
2022-02-16,5,0.0,40,0,-32.203390


In [141]:
display(len(larimer_vac_filled))
display(len(larimer_hosp_filled))
display(len(daily_cases_filled))
display(len(daily_deaths_filled))


430

690

1077

700

In [142]:
# valid_entries = larimer_vac.count()
# total_rows = len(larimer_vac.index)
# missing_data = total_rows - valid_entries
# missing_data


```python
merge_ordered(df1,
              df2,
              fill_method="ffill",
              on='column',
              how='outer'
```

- [x] Experimenting with merging on 'Date' column but it's been put back as an int instead of a datetime so may need to re-type that in all the DFs
- [x] Need to rename the date column in one of the frames so they can all be merged

In [143]:
# daily_cases_filled['Date'] = pd.to_datetime(daily_cases_filled['Date']).dt.tz_localize(None)
# daily_deaths_filled['Date'] = pd.to_datetime(daily_deaths_filled['Date']).dt.tz_localize(None)
# larimer_hosp_filled['Date'] = pd.to_datetime(larimer_hosp_filled['Date']).dt.tz_localize(None)
# larimer_vac_filled['Date'] = pd.to_datetime(larimer_vac_filled['Date']).dt.tz_localize(None)


In [144]:
death_case = pd.merge_ordered(
    daily_deaths_filled,
    daily_cases_filled,
    fill_method=None,
    on='Date',
    how='outer')

death_case


,Date,Unnamed: 0_x,death_id,death_date,age,gender,city,case_status,count,Unnamed: 0_y,CaseCount,ReportedDate,Sex,Age,Type,City
0,2020-03-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1,1,1,1
1,2020-03-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
2,2020-03-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
3,2020-03-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
4,2020-03-13,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,2023-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1073,2023-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1074,2023-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
1075,2023-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0


In [145]:
death_case_hosp = pd.merge_ordered(
    death_case,
    larimer_hosp_filled,
    fill_method=None,
    on='Date',
    how='outer')

death_case_hosp


,Date,Unnamed: 0_x,death_id,death_date,age,gender,city,case_status,count,Unnamed: 0_y,...,ReportedDate,Sex,Age,Type,City,admission_count,kpi_admits_indicator,inpatient_count,kpi_patient_indicator,inpatient_count_pct_change
0,2020-03-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,...,1,1,1,1,1,<NA>,NaN,NaN,NaN,NaN
1,2020-03-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
2,2020-03-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
3,2020-03-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
4,2020-03-13,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,2023-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
1073,2023-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
1074,2023-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN
1075,2023-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,<NA>,NaN,NaN,NaN,NaN


In [146]:
combo_df = pd.merge_ordered(
    death_case_hosp,
    larimer_vac_filled,
    fill_method=None,
    on='Date',
    how='outer')

combo_df


,Date,Unnamed: 0_x,death_id,death_date,age,gender,city,case_status,count,Unnamed: 0_y,...,daily number of Latinx residents vaccinated,total of Latinx residents vaccinated,daily number of White non-Latinx residents vaccinated,total of White non-Latinx residents vaccinated,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses
0,2020-03-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-13,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,2023-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1073,2023-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1074,2023-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1075,2023-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
for col in combo_df.columns:
    print(col)


Date
Unnamed: 0_x
death_id
death_date
age
gender
city
case_status
count
Unnamed: 0_y
CaseCount
ReportedDate
Sex
Age
Type
City
admission_count
kpi_admits_indicator
inpatient_count
kpi_patient_indicator
inpatient_count_pct_change
Unnamed: 0
daily number of doses received by Larimer County residents
total number of doses recevied by residents
daily number of residents receiving first dose
total number of residents receiving first dose
daily number of residents vaccinated
total number of residents vaccinated
daily number of 70+ vaccinated
total number of 70+ vaccinated
daily number of 70+ at least one dose
total number of 70+ at least one dose
daily number of Latinx residents vaccinated
total of Latinx residents vaccinated
daily number of White non-Latinx residents vaccinated
total of White non-Latinx residents vaccinated
daily number of non-White non-Latinx residents vaccinated
total of non-White non-Latinx residents vaccinated
dailyUnknown
totalUnknown
daily_additional_doses
total_additi

In [148]:
combo_df.rename(columns={'count': 'Daily Death Count',
                         'daily number of doses received by Larimer County residents': 'Daily doses',
                         'CaseCount': 'Daily Cases',
                         'admission_count': 'Daily Hospitalizations',
                         'inpatient_count': 'Total Patient Count'
                         }, inplace=True)
                         

In [149]:
combo_df[['Date', 'Daily doses', 'Daily Cases',
          'Daily Hospitalizations', 'Total Patient Count', 'Daily Death Count']]


,Date,Daily doses,Daily Cases,Daily Hospitalizations,Total Patient Count,Daily Death Count
0,2020-03-09,NaN,1,<NA>,NaN,1.0
1,2020-03-10,NaN,0,<NA>,NaN,0.0
2,2020-03-11,NaN,0,<NA>,NaN,0.0
3,2020-03-12,NaN,0,<NA>,NaN,0.0
4,2020-03-13,NaN,0,<NA>,NaN,1.0
...,...,...,...,...,...,...
1072,2023-02-14,NaN,0,<NA>,NaN,NaN
1073,2023-02-15,NaN,0,<NA>,NaN,NaN
1074,2023-02-16,NaN,0,<NA>,NaN,NaN
1075,2023-02-17,NaN,0,<NA>,NaN,NaN


In [150]:
print(combo_df.isna().sum().sum())


18538


In [151]:
combo_df = combo_df.fillna(0)
print(combo_df.isna().sum().sum())


0


In [152]:
combo_df[['Date', 'Daily doses', 'Daily Cases',
          'Daily Hospitalizations', 'Daily Death Count']]


,Date,Daily doses,Daily Cases,Daily Hospitalizations,Daily Death Count
0,2020-03-09,0.0,1,0,1.0
1,2020-03-10,0.0,0,0,0.0
2,2020-03-11,0.0,0,0,0.0
3,2020-03-12,0.0,0,0,0.0
4,2020-03-13,0.0,0,0,1.0
...,...,...,...,...,...
1072,2023-02-14,0.0,0,0,0.0
1073,2023-02-15,0.0,0,0,0.0
1074,2023-02-16,0.0,0,0,0.0
1075,2023-02-17,0.0,0,0,0.0


In [153]:
combo_df[['7-day avg doses',
          '7-day avg Cases',
          '7-day avg Hospitalizations']] = combo_df[['Daily doses',
                                              'Daily Cases',
                                              'Daily Hospitalizations']].rolling(7).mean().round(0)

combo_df[['7-Day avg Deaths']] = combo_df[['Daily Death Count']
                                          ].rolling(7).mean().round(3)


combo_df[['Date',
          'Daily doses',
          '7-day avg doses',
          'Daily Cases',
          '7-day avg Cases',
          'Daily Hospitalizations',
          '7-day avg Hospitalizations',
          'Daily Death Count',
          '7-Day avg Deaths',
          'Total Patient Count']]


,Date,Daily doses,7-day avg doses,Daily Cases,7-day avg Cases,Daily Hospitalizations,7-day avg Hospitalizations,Daily Death Count,7-Day avg Deaths,Total Patient Count
0,2020-03-09,0.0,NaN,1,NaN,0,NaN,1.0,NaN,0.0
1,2020-03-10,0.0,NaN,0,NaN,0,NaN,0.0,NaN,0.0
2,2020-03-11,0.0,NaN,0,NaN,0,NaN,0.0,NaN,0.0
3,2020-03-12,0.0,NaN,0,NaN,0,NaN,0.0,NaN,0.0
4,2020-03-13,0.0,NaN,0,NaN,0,NaN,1.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
1072,2023-02-14,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
1073,2023-02-15,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
1074,2023-02-16,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
1075,2023-02-17,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0


In [154]:
print(combo_df.isna().sum().sum())

24


In [155]:
combo_df = combo_df.fillna(0)
print(combo_df.isna().sum().sum())

0


In [156]:
# Make backup of final combo df
combo_df.to_csv('combo_df_backup.csv')
combo_df

,Date,Unnamed: 0_x,death_id,death_date,age,gender,city,case_status,Daily Death Count,Unnamed: 0_y,...,daily number of non-White non-Latinx residents vaccinated,total of non-White non-Latinx residents vaccinated,dailyUnknown,totalUnknown,daily_additional_doses,total_additional_doses,7-day avg doses,7-day avg Cases,7-day avg Hospitalizations,7-Day avg Deaths
0,2020-03-09,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-03-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-03-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-03-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-03-13,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1072,2023-02-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1073,2023-02-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1074,2023-02-16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1075,2023-02-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
combo_df[['Date',
          'Daily doses',
          '7-day avg doses',
          'Daily Cases',
          '7-day avg Cases',
          'Daily Hospitalizations',
          '7-day avg Hospitalizations',
          'Daily Death Count',
          '7-Day avg Deaths',
          'Total Patient Count']]

,Date,Daily doses,7-day avg doses,Daily Cases,7-day avg Cases,Daily Hospitalizations,7-day avg Hospitalizations,Daily Death Count,7-Day avg Deaths,Total Patient Count
0,2020-03-09,0.0,0.0,1,0.0,0,0.0,1.0,0.0,0.0
1,2020-03-10,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
2,2020-03-11,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
3,2020-03-12,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
4,2020-03-13,0.0,0.0,0,0.0,0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1072,2023-02-14,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
1073,2023-02-15,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
1074,2023-02-16,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0
1075,2023-02-17,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0


## Visualize

### Hover example

```python
hover = HoverTool()
hover.tooltips = [
    ("Totals", "@TONS_HE High Explosive / @TONS_IC Incendiary / @TONS_FRAG Fragmentation")]

hover.mode = 'vline'

p.add_tools(hover)

show(p)
```

### Color mapper

```python
#Use the field name of the column source
mapper = linear_cmap(field_name='y', palette=Spectral6 ,low=min(y) ,high=max(y))

source = ColumnDataSource(dict(x=x,y=y))

p = figure(width=300, height=300, title="Linear Color Map Based on Y")

p.circle(x='x', y='y', line_color=mapper,color=mapper, fill_alpha=1, size=12, source=source)

color_bar = ColorBar(color_mapper=mapper['transform'], width=8)

p.add_layout(color_bar, 'right')

```


### Saving Pattern

```python
output_notebook()

plot = figure(plot_width=600, plot_height=600, tools='pan,box_zoom,reset')

plot.square(x=[1, 2, 4, 8, 10], y=[6, 2, 18, 4, 9], size=20)


show(plot)
output_file('bokeh_save.html')
save(plot)
reset_output()
```

In [158]:
#Print the color palette we're using then flip it over turn it and reverse it and make sure it worked
print(RdBu[11])
BuRd = RdBu[11][::-1]
print(BuRd)


('#053061', '#2166ac', '#4393c3', '#92c5de', '#d1e5f0', '#f7f7f7', '#fddbc7', '#f4a582', '#d6604d', '#b2182b', '#67001f')
('#67001f', '#b2182b', '#d6604d', '#f4a582', '#fddbc7', '#f7f7f7', '#d1e5f0', '#92c5de', '#4393c3', '#2166ac', '#053061')


In [159]:
# Get and initialize the html template we are using to save our figures
env = Environment(loader=FileSystemLoader('templates'))
fig_template = env.get_template('fig_template.html')

In [160]:
output_notebook()

lar_vac_data = ColumnDataSource(combo_df)


TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

TOOLTIPS = [('Date', '@Date{ %F}'),
            ('Daily Doses', '@{Daily doses}'),
            ('7-day avg doses', '@{7-day avg doses}')]


mapper = linear_cmap(field_name='Daily doses',
                     palette=BuRd, low=(None), high=(None))

daily_vac_figure_combo = figure(title='Daily Vaccinations',
                                x_axis_type="datetime",
                                width=800,
                                tools=TOOLS)


daily_vac_figure_combo.vbar(x='Date',
                            top='Daily doses',
                            width=datetime.timedelta(days=0.5),
                            alpha=0.6,
                            color=mapper,
                            source=lar_vac_data)

line = daily_vac_figure_combo.line(x='Date',
                                   y='7-day avg doses',
                                   line_width=2,
                                   color='#000000',
                                   source=lar_vac_data)


vac_hover = HoverTool()

vac_hover.tooltips = TOOLTIPS

vac_hover.mode = 'vline'

vac_hover.formatters = {'@Date': 'datetime'}

vac_hover.renderers = [line]

daily_vac_figure_combo.add_tools(vac_hover)

#daily_vac_figure_combo.add_tools(HoverTool(tooltips=[("Date", "@x"),('Doses','@top')]))


Loading BokehJS ...

In [161]:
show(daily_vac_figure_combo)
save(daily_vac_figure_combo, title='Vaccination Plot', filename='larimer_fig_docs/vac_mod.html',template=fig_template)
reset_output()

/Users/cstephenson/opt/anaconda3/envs/updated_jupyter/lib/python3.9/site-packages/bokeh/io/saving.py:142: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")


This was my attempt at a js soclution

```python
js, vac_tag = autoload_static(daily_vac_figure_combo, CDN, "fig_embeds/vac.js") # path doesn't matter; won't be using
    
file = open('fig_embeds/vac.js', 'w') # filename doesn't matter either, use whatever you like
file.write(js)
file.close()
print(vac_tag)
```

In [162]:
output_notebook()

lar_vac_data = ColumnDataSource(combo_df)


TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

TOOLTIPS = [('Date', '@Date{ %F}'),
            ('Daily Cases', '@{Daily Cases}'),
            ('7-day avg Cases', '@{7-day avg Cases}')]


mapper = linear_cmap(field_name='Daily Cases', palette=list(
    reversed(cc.fire)), low=(None), high=(None))

daily_case_figure_combo = figure(title='Daily Cases',
                                 x_axis_type="datetime",
                                 width=800,
                                 tools=TOOLS)


daily_case_figure_combo.vbar(x='Date',
                             top='Daily Cases',
                             width=datetime.timedelta(days=0.5),
                             alpha=0.8,
                             color=mapper,
                             source=lar_vac_data)


line = daily_case_figure_combo.line(x='Date',
                                    y='7-day avg Cases',
                                    line_width=2,
                                    color='#000000',
                                    source=lar_vac_data)


case_hover = HoverTool()

case_hover.tooltips = TOOLTIPS

case_hover.mode = 'vline'

case_hover.formatters = {'@Date': 'datetime'}

case_hover.renderers = [line]

daily_case_figure_combo.add_tools(case_hover)


Loading BokehJS ...

In [163]:
show(daily_case_figure_combo)
save(daily_case_figure_combo, title='Daily Case Plot', filename='larimer_fig_docs/case_mod.html',template=fig_template)
reset_output()

In [164]:
output_notebook()

lar_vac_data = ColumnDataSource(combo_df)

TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

TOOLTIPS = [('Date', '@Date{ %F}'),
            ('Daily Hospitalizations', '@{Daily Hospitalizations}'),
            ('7-day avg Hospitalizations', '@{7-day avg Hospitalizations}')]


mapper = linear_cmap(field_name='Daily Hospitalizations',
                     palette=list(reversed(cc.fire)), low=(None), high=(None))

daily_hosp_figure_combo = figure(title='Daily Hospitalizations',
                                 x_axis_type="datetime",
                                 width=800,
                                 tools=TOOLS)


daily_hosp_figure_combo.vbar(x='Date',
                             top='Daily Hospitalizations',
                             width=datetime.timedelta(days=0.5),
                             alpha=0.5,
                             color=mapper,
                             source=lar_vac_data)


line = daily_hosp_figure_combo.line(x='Date',
                                    y='7-day avg Hospitalizations',
                                    line_width=2,
                                    color='#000000',
                                    source=lar_vac_data)


hosp_hover = HoverTool()

hosp_hover.tooltips = TOOLTIPS

hosp_hover.mode = 'vline'

hosp_hover.formatters = {'@Date': 'datetime'}

hosp_hover.renderers = [line]

daily_hosp_figure_combo.add_tools(hosp_hover)


Loading BokehJS ...

In [165]:
show(daily_hosp_figure_combo)
save(daily_hosp_figure_combo, title='Hospitalization Plot', filename='larimer_fig_docs/hosp_mod.html',template=fig_template)
reset_output()

In [166]:
output_notebook()

lar_vac_data = ColumnDataSource(combo_df)


TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

TOOLTIPS = [('Date', '@Date{ %F}'),
            ('Total Patient Count', '@{Total Patient Count}')]


mapper = linear_cmap(field_name='Total Patient Count', palette=list(
    reversed(cc.fire)), low=(None), high=(None))

total_patient_figure = figure(title='Total Patient Count',
                              x_axis_type="datetime",
                              width=800,
                              tools=TOOLS)


total_patient_figure.vbar(x='Date',
                          top='Total Patient Count',
                          width=datetime.timedelta(days=0.5),
                          alpha=0.9,
                          color=mapper,
                          source=lar_vac_data)

pat_line = total_patient_figure.line(x='Date',
                                     y='Total Patient Count',
                                     alpha=0.9,
                                     source=lar_vac_data)


patient_hover = HoverTool()

patient_hover.tooltips = TOOLTIPS

patient_hover.mode = 'vline'

patient_hover.formatters = {'@Date': 'datetime'}

patient_hover.renderers = [pat_line]

total_patient_figure.add_tools(patient_hover)


Loading BokehJS ...

In [167]:
show(total_patient_figure)
save(total_patient_figure, title='Patient Plot', filename='larimer_fig_docs/patient_mod.html',template=fig_template)
reset_output()

In [168]:
output_notebook()

lar_vac_data = ColumnDataSource(combo_df)

reset_output()
output_notebook()

TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

TOOLTIPS = [('Date', '@Date{ %F}'),
            ('Daily Deaths', '@{Daily Death Count}'),
            ('7-day avg Deaths', '@{7-Day avg Deaths}')]


mapper = linear_cmap(field_name='Daily Death Count', palette=list(
    reversed(cc.fire)), low=(None), high=(None))

daily_death_figure_combo = figure(title='Daily Deaths',
                                  x_axis_type="datetime",
                                  width=800,
                                  tools=TOOLS)


daily_death_figure_combo.vbar(x='Date',
                              top='Daily Death Count',
                              width=datetime.timedelta(days=0.5),
                              alpha=0.8,
                              color=mapper,
                              source=lar_vac_data)


line = daily_death_figure_combo.line(x='Date',
                                     y='7-Day avg Deaths',
                                     line_width=2,
                                     color='#000000',
                                     source=lar_vac_data)


death_hover = HoverTool()

death_hover.tooltips = TOOLTIPS

death_hover.mode = 'vline'

death_hover.formatters = {'@Date': 'datetime'}

death_hover.renderers = [line]

daily_death_figure_combo.add_tools(death_hover)

Loading BokehJS ...

Loading BokehJS ...

In [169]:
show(daily_death_figure_combo)
save(daily_death_figure_combo, title='Death Plot', filename='larimer_fig_docs/death_mod.html',template=fig_template)
reset_output()

# BOOKMARK


- [ ] Get overall plot layout


## Combo Figure layout

### Layout example

```python
show(row(column(fig1, fig2), column(fig3)))
```

In [170]:
output_notebook()


#show(row(column(daily_vac_figure_combo, daily_case_figure_combo), column(daily_death_figure_combo, daily_hosp_figure_combo)))


Loading BokehJS ...

In [171]:
# output_notebook()

# lar_vac_data = ColumnDataSource(combo_df)


# TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

# TOOLTIPS = [('Date', '@Date{ %F}'),
#             ('Daily Doses','@{Daily doses}'),
#             ('7-day avg doses','@{7-day avg doses}')]


# blue_mapper = linear_cmap(field_name='Daily doses', palette=BuRd, low=(None), high=(None))
# red_mapper =  linear_cmap(field_name='Daily Cases', palette=list(reversed(cc.fire)), low=(None), high=(None))

# daily_vac_figure_combo = figure(title='Daily Vaccinations',
#                             x_axis_type="datetime",
#                             tools=TOOLS)


# daily_vac_figure_combo.vbar(x='Date',
#                           top='Daily doses',
#                           width=datetime.timedelta(days=0.5),
#                           alpha=0.3,
#                           color=blue_mapper,
#                           source=lar_vac_data)

# line = daily_vac_figure_combo.line(x='Date',
#                                    y='7-day avg doses',
#                                    line_width=2,
#                                    color='#000000',
#                                    source=lar_vac_data)


# vac_hover = HoverTool()

# vac_hover.tooltips = TOOLTIPS

# vac_hover.mode = 'vline'

# vac_hover.formatters = {'@Date' : 'datetime'}

# vac_hover.renderers = [line]

# daily_vac_figure_combo.add_tools(vac_hover)


# show(daily_vac_figure_combo)
# output_file('vac_figure.html')
# save(daily_vac_figure_combo)
# reset_output()

# output_notebook()

# lar_vac_data = ColumnDataSource(combo_df)


# TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

# TOOLTIPS = [('Date', '@Date{ %F}'),
#             ('Daily Cases','@{Daily Cases}'),
#             ('7-day avg Cases','@{7-day avg Cases}')]


# mapper = linear_cmap(field_name='Daily Cases', palette=list(reversed(cc.fire)), low=(None), high=(None))

# daily_case_figure_combo = figure(title='Daily Cases',
#                               x_axis_type="datetime",
#                               width=800,
#                               tools=TOOLS)


# daily_case_figure_combo.vbar(x='Date',
#                           top='Daily Cases',
#                           width=datetime.timedelta(days=0.5),
#                           alpha=0.8,
#                           color=mapper,
#                           source=lar_vac_data)


# line = daily_case_figure_combo.line(x='Date',
#                       y='7-day avg Cases',
#                       line_width=2,
#                       color='#000000',
#                       source=lar_vac_data)


# case_hover = HoverTool()

# case_hover.tooltips = TOOLTIPS

# case_hover.mode = 'vline'

# case_hover.formatters = {'@Date' : 'datetime'}

# case_hover.renderers = [line]

# daily_case_figure_combo.add_tools(case_hover)


# show(daily_case_figure_combo)
# output_file('case_figure.html')
# save(daily_case_figure_combo)
# reset_output()


# output_notebook()

# lar_vac_data = ColumnDataSource(combo_df)

# TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

# TOOLTIPS = [('Date', '@Date{ %F}'),
#             ('Daily Hospitalizations','@{Daily Hospitalizations}'),
#             ('7-day avg Hospitalizations','@{7-day avg Hospitalizations}')]


# mapper = linear_cmap(field_name='Daily Hospitalizations', palette=list(reversed(cc.fire)), low=(None), high=(None))

# daily_hosp_figure_combo = figure(title='Daily Hospitalizations',
#                               x_axis_type="datetime",
#                               width=800,
#                               tools=TOOLS)


# daily_hosp_figure_combo.vbar(x='Date',
#                           top='Daily Hospitalizations',
#                           width=datetime.timedelta(days=0.5),
#                           alpha=0.5,
#                           color=mapper,
#                           source=lar_vac_data)


# line = daily_hosp_figure_combo.line(x='Date',
#                       y='7-day avg Hospitalizations',
#                       line_width=2,
#                       color='#000000',
#                       source=lar_vac_data)


# hosp_hover = HoverTool()

# hosp_hover.tooltips = TOOLTIPS

# hosp_hover.mode = 'vline'

# hosp_hover.formatters = {'@Date' : 'datetime'}

# hosp_hover.renderers = [line]

# daily_hosp_figure_combo.add_tools(hosp_hover)

# show(daily_hosp_figure_combo)
# output_file('hosp_figure.html')
# save(daily_hosp_figure_combo)
# reset_output()


# output_notebook()

# lar_vac_data = ColumnDataSource(combo_df)


# TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

# TOOLTIPS = [('Date', '@Date{ %F}'),
#             ('Total Patient Count','@{Total Patient Count}')]


# mapper = linear_cmap(field_name='Total Patient Count', palette=list(reversed(cc.fire)), low=(None), high=(None))

# total_patient_figure = figure(title='Total Patient Count',
#                               x_axis_type="datetime",
#                               width=800,
#                               tools=TOOLS)


# total_patient_figure.vbar(x='Date',
#                                top='Total Patient Count',
#                                width=datetime.timedelta(days=0.5),
#                                alpha=0.9,
#                                color=mapper,
#                                source=lar_vac_data)

# pat_line = total_patient_figure.line(x='Date',
#                                y='Total Patient Count',
#                                alpha=0.9,
#                                source=lar_vac_data)


# patient_hover = HoverTool()

# patient_hover.tooltips = TOOLTIPS

# patient_hover.mode = 'vline'

# patient_hover.formatters = {'@Date' : 'datetime'}

# patient_hover.renderers = [pat_line]

# total_patient_figure.add_tools(patient_hover)


# show(total_patient_figure)
# output_file('patient_total_figure.html')
# save(total_patient_figure)
# reset_output()

# output_notebook()

# lar_vac_data = ColumnDataSource(combo_df)

# reset_output()
# output_notebook()

# TOOLS = 'pan, wheel_zoom, box_zoom, reset, save'

# TOOLTIPS = [('Date', '@Date{ %F}'),
#             ('Daily Deaths','@{Daily Death Count}'),
#             ('7-day avg Deaths','@{7-Day avg Deaths}')]


# mapper = linear_cmap(field_name='Daily Death Count', palette=list(reversed(cc.fire)), low=(None), high=(None))

# daily_death_figure_combo = figure(title='Daily Deaths',
#                               x_axis_type="datetime",
#                               width=800,
#                               tools=TOOLS)


# daily_death_figure_combo.vbar(x='Date',
#                           top='Daily Death Count',
#                           width=datetime.timedelta(days=0.5),
#                           alpha=0.8,
#                           color=mapper,
#                           source=lar_vac_data)


# line = daily_death_figure_combo.line(x='Date',
#                       y='7-Day avg Deaths',
#                       line_width=2,
#                       color='#000000',
#                       source=lar_vac_data)


# death_hover = HoverTool()

# death_hover.tooltips = TOOLTIPS

# death_hover.mode = 'vline'

# death_hover.formatters = {'@Date' : 'datetime'}

# death_hover.renderers = [line]

# daily_death_figure_combo.add_tools(death_hover)


# show(daily_death_figure_combo)
# output_file('deaths_figure.html')
# save(daily_death_figure_combo)
# reset_output()
